In [1]:
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np
import seaborn as sns
from IPython.display import HTML

sns.set()

## Exercise 5
**Determine  the  equation  of  motion  for  a simple pendulum of length `l` swinging through an arc in the x, y plane from an initial angle of Θ.**

For this exercise, I will assume that the fixed end of the pendulum is located at `(x, y) = (0, l)`. This makes the animation in Matplotlib easier. 

$$
\begin{align}
x &= l \cdot sin(\theta) \\
y &= l \cdot (1-cos\theta)
\end{align}
$$

Also, 
$$
\begin{align}
\dot x &= l \cdot sin\theta \cdot {\dot \theta} \\
\dot y &= l \cdot cos\theta \cdot {\dot \theta}
\end{align}
$$

Potential energy is just $V(x, y) = V(y) = mg(l-l cos\theta)$. Here, I have assumed that at `y = 0`, height is 0, hence potential energy then is 0.

Thus, the Lagrangian is given by,
$$
L = \frac{ml^2 {\dot{\theta}}^2}{2} - mg(l-lcos\theta)
$$

Sovling the Euler-Lagrangian equation, 
$$
\begin{align*}
l^2 {\ddot \theta} &= -mglsin\theta \\
\ddot \theta &= -\frac{m g sin\theta}{l}
\end{align*}
$$

Following simulation clarifies the ideas.

In [2]:
g = 9.81

"""
When an object of State is created, it is assumed that 
initial velocity is 0.
"""
class State:
    def __init__(self, theta0, m, l):
        self.theta = theta0
        self.mg_by_l = (g * m)/l
        self.m = m
        self.l = l
        self.v = 0
            
    def acceleration(self):
        """Returns accleration at current theta."""
        return -self.mg_by_l * np.sin(self.theta)
    
    def step(self):
        a = self.acceleration()
        self.v += 0.01 * a
        self.theta += 0.01 * self.v
        
    def pos(self):
        return self.l*np.sin(self.theta), self.l*(1-np.cos(self.theta))

In [3]:
def canvas(state):
    fig = plt.figure(figsize=(5,5))
    fig.suptitle("Mass = %.2f" % state.m)
    fig.gca().set_aspect('equal', adjustable='box')
    ax = plt.subplot(111)
    ax.set_xlim((-state.l, state.l)); ax.set_xlabel('X')
    ax.set_ylim((0, state.l+.5)); ax.set_ylabel('Y')
    x, y = state.pos()
    plt.plot([0], [state.l], 'b', ms=20)
    pt, = plt.plot([x], [y], 'g.', ms=20)
    line, = plt.plot([0, x], [state.l, y], 'g-')
    return fig, pt, line

def drawframe(n):
    state.step()
    x, y = state.pos()
    pt.set_data([x], [y])
    line.set_data([0, x], [state.l, y])
    return (pt, line)

In [4]:
state = State(theta0=np.pi/4, m=1, l=1)
fig, pt, line = canvas(state)
anim = animation.FuncAnimation(fig, drawframe, frames=200, interval=30, blit=True)
html = anim.to_html5_video()
plt.close()
HTML(html)

In [5]:
state = State(theta0=np.pi/4, m=10, l=1)
fig, pt, line = canvas(state)
anim = animation.FuncAnimation(fig, drawframe, frames=200, interval=30, blit=True)
html = anim.to_html5_video()
plt.close()
HTML(html)

## Exercise 6

I'll also assume, unlike previous exercise, that y increases downwards. I'll invert the axis in the animation to play nicely with Matplotlib.

Also, I don't have energy to typeset the derivation, so just posting the screenshot of my whiteboard. You probably will have hard time reading this (probably I too will have hard time reading it after couple of days).

![derivation](images/lect7_ex6.jpg)

Basically, we have two euler-lagrangian equations that are linear, and two unknowns $[\ddot \theta, \ddot \alpha]$. I probably could have had solved them on paper, but I took a shortcut and let Numpy solve it for me.

For the animation below, I've assumed that $l_1 = m_1 = l_2 = m_2 = 0$.

In [6]:
from numpy import cos, sin

class State:
    def __init__(self, theta0, alpha0):
        self.theta = theta0
        self.alpha = alpha0
        self.dtheta = 0  # theta_dot
        self.dalpha = 0 # alpha_dot
            
    def acceleration(self):
        """Returns accleration at current theta.
        Form a matrix M,
        
        [[p, q],
         [q, 1]]
         
        and a vector B = [u, v] s.t. M @ [ddtheta, ddalpha] = B.
        """
        t = self.theta
        a = self.alpha
        da = self.dalpha
        dt = self.dtheta
        
        sina, cosa = sin(a), cos(a)
        sinat = sin(a+t)
        
        p = 3 + 2*cosa
        q = 1 + cosa
        M = np.array([[p, q], [q, 1]])
        
        
        u = da*da*sina + 2*da*dt*sina - g*(2*sin(t) + sinat)
        v = dt*dt*sina - g*sinat
        
        ddt, dda = np.linalg.solve(M, np.array([u, v]))
        return ddt, dda
        
    def step(self):
        ddt, dda = self.acceleration()
        self.dtheta += 0.01 * ddt
        self.dalpha += 0.01 * dda
        self.theta += 0.01 * self.dtheta
        self.alpha += 0.01 * self.dalpha
        
    def pos(self):
        a, t = self.alpha, self.theta
        x1, y1 = sin(t), cos(t)
        x2, y2 = x1+sin(a+t), y1+cos(a+t)
        return (x1, -y1), (x2, -y2)

In [7]:
def canvas(state):
    fig = plt.figure(figsize=(5,5))
    fig.gca().set_aspect('equal', adjustable='box')
    ax = plt.subplot(111)
    ax.set_xlim((-2, 2)); ax.set_xlabel('X')
    ax.set_ylim((-2, 0.5)); ax.set_ylabel('Y')
    
    (x1, y1), (x2, y2) = state.pos()
    p1, = plt.plot([x1], [y1], 'g.', ms=20)
    p2, = plt.plot([x2], [y2], 'b.', ms=20)
    
    line1, = plt.plot([0, x1], [0, y1], 'g-')
    line2, = plt.plot([x1, x2], [y1, y2], 'b-')
    return fig, p1, p2, line1, line2

def drawframe(n):
    state.step()
    (x1, y1), (x2, y2) = state.pos()
    pt1.set_data([x1], [y1])
    pt2.set_data([x2], [y2])
    line1.set_data([0, x1], [0, y1])
    line2.set_data([x1, x2], [y1, y2])
    return pt1, pt2, line1, line2

In [8]:
state = State(np.pi/4, np.pi/6)
fig, pt1, pt2, line1, line2 = canvas(state)
anim = animation.FuncAnimation(fig, drawframe, frames=270, interval=30, blit=True)
html = anim.to_html5_video()
plt.close()
HTML(html)

Q: How did I know how many frames to compute, for smooth animation?

A: I didn't. I used grid search (aka trial and error) to figure out correct number of frames.